# **Collaborative Filtering**

## Objectives

- Create recommendation system based on collaborative filtering

**Recommendation systems** are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous and can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore recommendation systems based on Collaborative Filtering and implement simple version of one using Python and the Pandas library.

## Import the required Libraries

In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Loading the Dataset

In [2]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

## Understand the data

In [3]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169.0,2.5,1.204928e+09
1,1,2471.0,3.0,1.204927e+09
2,1,48516.0,5.0,1.204927e+09
3,2,2571.0,3.5,1.436165e+09
4,2,109487.0,4.0,1.436165e+09


## Data Pre-processing

1. Remove the year from the title column by using pandas' replace function and store in a new year column.


In [7]:
# Using regular expressions to find a year stored between parentheses
# We specify the parentheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df['title'].str.extract('\((\d{4})\)', expand=False)
# Removing the parentheses and the years from the 'title' column
movies_df['title'] = movies_df['title'].str.replace('\(\d{4}\)', '')
# Applying the strip function to get rid of any leading or trailing whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.split('(')[0].strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


2. With that, let's also drop the genres column since we won't need it for this particular recommendation system.

In [8]:
# Dropping the genres column
movies_df = movies_df.drop(columns='genres')

In [9]:
# Final Dataframe
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


Next, let's look at the ratings dataframe

3. Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.

In [10]:
# Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop(columns='timestamp')

In [11]:
# Final Dataframe
ratings_df.head()

,userId,movieId,rating
0,1,169.0,2.5
1,1,2471.0,3.0
2,1,48516.0,5.0
3,2,2571.0,3.5
4,2,109487.0,4.0


# **Collaborative Filtering**
Now it's time to start our work on recommendation systems.

The first technique we're going to take a look at is called Collaborative Filtering, which is also known as User-User Filtering. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the Pearson Correlation Function.

The process for creating a User Based recommendation system is as follows:

- Select a user with the movies the user has watched
- Based on his rating of the movies, find the top X neighbours
- Get the watched movie record of the user for each neighbour
- Calculate a similarity score using some formula
- Recommend the items with the highest score

Let's begin by creating an input user to recommend movies to:

In [12]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ]
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


Add movieId to input user

With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [14]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop(columns='year')
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


The users who has seen the same movies
Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.

In [15]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
19,4,296.0,4.0
441,12,1968.0,3.0
479,13,2.0,2.0
531,13,1274.0,5.0
681,14,296.0,2.0


group up the rows by user ID

In [16]:
userSubsetGroup = userSubset.groupby(['userId'])

In [17]:
# Check one of the users randomly
userSubsetGroup.get_group(1130)

,userId,movieId,rating
104167,1130,1.0,0.5
104168,1130,2.0,4.0
104214,1130,296.0,4.0
104363,1130,1274.0,4.5
104443,1130,1968.0,4.5


Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.

In [18]:
# Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

Now let's look at the first user

In [19]:
userSubsetGroup[0:3]

[((75,),
        userId  movieId  rating
  7507      75      1.0     5.0
  7508      75      2.0     3.5
  7540      75    296.0     5.0
  7633      75   1274.0     4.5
  7673      75   1968.0     5.0),
 ((106,),
        userId  movieId  rating
  9083     106      1.0     2.5
  9084     106      2.0     3.0
  9115     106    296.0     3.5
  9198     106   1274.0     3.0
  9238     106   1968.0     3.5),
 ((686,),
         userId  movieId  rating
  61336     686      1.0     4.0
  61337     686      2.0     3.0
  61377     686    296.0     4.0
  61478     686   1274.0     4.0
  61569     686   1968.0     5.0)]

## Similarity of users to input user
Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.  
We're going to find out how similar each user is to the input through the __Pearson Correlation Coefficient__. It is used to measure the strength of a linear association between the two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below.

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y, then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because, for example, two users might rate two series of items totally differently in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation.

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.


We will select a subset of users to iterate through. This limit is imposed because we don't want to waste too much time going through every single user.

In [20]:
userSubsetGroup = userSubsetGroup[0:100]

Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient.

In [21]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [22]:
pearsonCorrelationDict.items()

dict_items([((75,), 0.8272781516947562), ((106,), 0.5860090386731182), ((686,), 0.8320502943378437), ((815,), 0.5765566601970551), ((1040,), 0.9434563530497265), ((1130,), 0.2891574659831201), ((1502,), 0.8770580193070299), ((1599,), 0.4385290096535153), ((1625,), 0.716114874039432), ((1950,), 0.179028718509858), ((2065,), 0.4385290096535153), ((2128,), 0.5860090386731196), ((2432,), 0.1386750490563073), ((2791,), 0.8770580193070299), ((2839,), 0.8204126541423674), ((2948,), -0.11720180773462392), ((3025,), 0.45124262819713973), ((3040,), 0.89514359254929), ((3186,), 0.6784622064861935), ((3271,), 0.26989594817970664), ((3429,), 0.0), ((17,), -0.3458572319330373), ((114,), 0.6363636363636364), ((122,), 0.24902912254587614), ((217,), 0.8819171036881969), ((222,), 0), ((241,), -0.3799802978286742), ((277,), 0.8703882797784892), ((340,), 0.8703882797784892), ((373,), 0.723746864455746), ((393,), 0.592156525463792), ((407,), 0.6446583712203042), ((432,), 0.8528028654224418), ((447,), 0.522

In [23]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.827278,"(75,)"
1,0.586009,"(106,)"
2,0.832050,"(686,)"
3,0.576557,"(815,)"
4,0.943456,"(1040,)"


The top x similar users to input user

Now let's get the top 50 users that are most similar to the input.

In [24]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
53,1.000000,"(1255,)"
39,0.986928,"(599,)"
55,0.971061,"(1519,)"
42,0.965581,"(784,)"
48,0.965060,"(1066,)"


Now, let's start recommending movies to the input user.

Rating of selected users to all movies
We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our pearsonDF from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [30]:
# Extract individual integer values from tuples in 'userId' column
topUsers['userId'] = topUsers['userId'].apply(lambda x: x[0] if isinstance(x, tuple) else x)

# Convert 'userId' column to int64 type
topUsers['userId'] = topUsers['userId'].astype(int)

# Now you can merge the DataFrames
topUsersRating = pd.merge(topUsers, ratings_df, on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,1.0,1255,1.0,3.0
1,1.0,1255,2.0,2.5
2,1.0,1255,10.0,3.5
3,1.0,1255,18.0,3.5
4,1.0,1255,19.0,3.5


Now all we need to do is simply multiply the movie rating by its weight (the similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:

In [31]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,1255,1.0,3.0,3.0
1,1.0,1255,2.0,2.5,2.5
2,1.0,1255,10.0,3.5,3.5
3,1.0,1255,18.0,3.5,3.5
4,1.0,1255,19.0,3.5,3.5


In [32]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1.0,41.098879,155.978682
2.0,41.077298,99.329598
3.0,10.386534,29.685825
4.0,2.299245,6.522353
5.0,11.256188,31.087047


In [33]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1.0,3.795205,1.0
2.0,2.418114,2.0
3.0,2.858107,3.0
4.0,2.836737,4.0
5.0,2.761774,5.0


Now let's sort it and see the top 20 movies that the algorithm recommended!

In [34]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
934.0,5.0,934.0
91233.0,5.0,91233.0
326.0,5.0,326.0
5298.0,5.0,5298.0
3284.0,5.0,3284.0
3318.0,5.0,3318.0
91488.0,5.0,91488.0
3329.0,5.0,3329.0
5088.0,5.0,5088.0


In [35]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
322,326,To Live,1994
917,934,Father of the Bride,1950
3198,3284,They Might Be Giants,1971
3232,3318,Deterrence,1999
3243,3329,"Year My Voice Broke, The",1987
4993,5088,Going Places,1974
5202,5298,Human Nature,2001
6343,6452,"Long, Hot Summer, The",1958
18285,91233,Lifted,2006
18343,91488,"Snowman, The",1982


# Advantages and Disadvantages of Collaborative Filtering
## Advantages
- Takes other user's ratings into consideration
- Doesn't need to study or extract information from the recommended item
- Adapts to the user's interests which might change over time

## Disadvantages
- Approximation function can be slow
- There might be a low amount of users to approximate
- Privacy issues when trying to learn the user's preferences